# Рекомендация тарифов

Оператор мобильной связи выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра». В нашем распоряжении данные о поведении клиентов, которые уже перешли на новые тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф, для клиентов пользующихся архивными тарифами.

# Описание данных

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
    
сalls — количество звонков,

minutes — суммарная длительность звонков в минутах,

messages — количество sms-сообщений,

mb_used — израсходованный интернет-трафик в Мб,

is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## Откройте и изучите файл

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

In [2]:
try:
    data = pd.read_csv("users_behavior.csv")
except:
    data = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
data.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [5]:
data['calls'] = data['calls'].astype('int')
data['messages'] = data['messages'].astype('int')

### Колличество звонков и сообщений не может быть не целое число

In [6]:
data.isna().sum()

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

In [7]:
data.duplicated().sum()

0

### Данные импортированы и проверены, повторов и пропусков нет, столбцы названы корректно, типы колонок "звонков" и "сообщений" были изменены в соответствии с типом хранящихся в них данных

## Разбейте данные на выборки

### Разделим набор данных на обучающую (train(60%)), тестовые(test(20%)) и валидационную (validate(20%)) выборки. В random_state положим значение 12345

In [8]:
train, validate, test = \
              np.split(data.sample(frac=1, random_state=42), 
                       [int(.6*len(data)), int(.8*len(data))])

In [9]:
features_train = train.drop(['is_ultra'], axis=1)
target_train = train['is_ultra']
features_valid = validate.drop(['is_ultra'], axis=1)
target_valid = validate['is_ultra']
test_features = test.drop(['is_ultra'], axis=1)
test_target = test['is_ultra']

In [10]:
# Код ревьюера
features_train.shape, features_valid.shape, test_features.shape

((1928, 4), (643, 4), (643, 4))

## Исследуйте модели

### Для начала попробуем модель поэксперементировать с решающим деревом оно подходит для задач классификации

In [11]:
best_model = None
best_result = 0
for depth in range(1, 6):
    model_tree = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model_tree.fit(features_train, target_train)
    predictions = model_tree.predict(features_valid)
    result = accuracy_score(target_valid, predictions)
    if result > best_result:
        best_model = model_tree
        best_result = result
print("Accuracy лучшей модели:", best_result)

Accuracy лучшей модели: 0.7853810264385692


#### Поставим глубину дерева на 10

In [12]:
model_tree = DecisionTreeClassifier(random_state=12345, max_depth=10)
model_tree.fit(features_train, target_train)
predictions = model_tree.predict(features_valid)
result = accuracy_score(target_valid, predictions)
print('Точность', result)

Точность 0.7916018662519441


**Лучшая глубина дерева оказалась 10, а лучшая точность 0.791**

#### Настроим гиперпараметр criterion с gini на entropy

In [13]:
model_tree = DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=depth, random_state=12345, 
                                    max_features=None, max_leaf_nodes=None, min_impurity_decrease=0.0, 
                                    min_impurity_split=None, min_samples_leaf=1, min_samples_split=2, 
                                    min_weight_fraction_leaf=0.0, splitter='best') 
model_tree.fit(features_train, target_train)
predictions = model_tree.predict(features_valid)
result = accuracy_score(target_valid, predictions)
print('Точность', result)

Точность 0.7993779160186625


#### Настроим гиперпараметр splitter с best на random

In [14]:
model = DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=depth, random_state=12345, 
                                    max_features=None, max_leaf_nodes=None, min_impurity_decrease=0.0, 
                                    min_impurity_split=None, min_samples_leaf=1, min_samples_split=2, 
                                    min_weight_fraction_leaf=0.0, splitter='random') 
model.fit(features_train, target_train)
predictions = model.predict(features_valid)
result = accuracy_score(target_valid, predictions)
print('Точность', result)

Точность 0.7916018662519441


### Теперь попробуем случайный лес

In [15]:
best_model = None
best_result = 0
best_depth = 0

for est in range(1, 11):
    model_random = RandomForestClassifier(random_state=12345, n_estimators=est)
    model_random.fit(features_train, target_train)
    result = model_random.score(features_valid, target_valid)
    if result > best_result:
        best_model = model_random
        best_result = result
        best_depth = depth

print('Лучшая количество деревьев', best_depth,'Лучшая точность', best_result)

Лучшая количество деревьев 5 Лучшая точность 0.7931570762052877


**Чуть точнее, но не сильно 0.793**

#### Настроим колличество деревьев

In [16]:
model_random = RandomForestClassifier(random_state=12345, n_estimators=20)
model_random.fit(features_train, target_train)
predictions = model_random.predict(features_valid)
result = model_random.score(features_valid, target_valid)
print('Точность', result)

Точность 0.8040435458786936


**0.8 уже лучше**

#### Настроим гиперпараметр criterion с gini на entropy

In [17]:
model_random = RandomForestClassifier(random_state=12345, n_estimators=20, criterion='entropy')
model_random.fit(features_train, target_train)
predictions = model_random.predict(features_valid)
result = model_random.score(features_valid, target_valid)
print('Точность', result)

Точность 0.8102643856920684


**Идём на повышение**

#### Настроим гиперпараметр oob_score с False на True

In [18]:
model_random = RandomForestClassifier(random_state=12345, n_estimators=20, criterion='entropy', oob_score=True)
model_random.fit(features_train, target_train)
predictions = model_random.predict(features_valid)
result = model_random.score(features_valid, target_valid)
print('Точность', result)

Точность 0.8102643856920684


### Перейдём к логистической регрессии

In [19]:
best_model = None
best_result = 0
best_max_iter = 0

for max_iter in range(1,50):
    model_log = LogisticRegression(random_state=12345, max_iter=1000)
    model_log.fit(features_train, target_train)
    predictions = model_log.predict(features_valid)
    result = model_log.score(features_valid, target_valid)
    if result > best_result:
        best_model = model_log
        best_result = result
        best_max_iter = max_iter

print('Лучшая количество итерраций', best_max_iter,'Лучшая точность', best_result)        

Лучшая количество итерраций 1 Лучшая точность 0.7013996889580093


**0.70, не хорошо, посмотрим что получится если покрутить настройки**

#### Настроим гиперпараметр multi_class с auto на multinomia

In [20]:
model_log = LogisticRegression(random_state=12345, max_iter=1000, multi_class='multinomial')
model_log.fit(features_train, target_train)
predictions = model_log.predict(features_valid)
result = model_log.score(features_valid, target_valid)
print('Точность', result)

Точность 0.7325038880248833


#### Изменим гиперпараметр warm_start на True

In [21]:
model_log = LogisticRegression(random_state=12345, max_iter=1000, multi_class='multinomial', warm_start=True)
model_log.fit(features_train, target_train)
predictions = model_log.predict(features_valid)
result = model_log.score(features_valid, target_valid)
print('Точность', result)

Точность 0.7325038880248833


#### Проверим гипернастройку solver и изменим её на lbfgs

In [22]:
model_log = LogisticRegression(random_state=12345, max_iter=1000, multi_class='multinomial', warm_start=True, solver='lbfgs')
model_log.fit(features_train, target_train)
predictions = model_log.predict(features_valid)
result = model_log.score(features_valid, target_valid)
print('Точность', result)

Точность 0.7325038880248833


### В результате исследования данных были полученые следующие результаты:

**Решающие древо - 0.79**

**Случайны лес - 0.81**

**Логическая регрессия - 0.73**

**Случайный лес победил!**

## Проверьте модель на тестовой выборке

### Применим модель случайного леса на тестовой выборке, что бы убедится.

In [23]:
test_predictions = model_random.predict(test_features)
accuracy = accuracy_score(test_target, test_predictions)
print('Точность модели случайного леса на тестовой выборке',accuracy)

Точность модели случайного леса на тестовой выборке 0.7978227060653188


**Самая лучшая модель - это случайный лес, она показывает самые высокие показатели точности.** 

## Проверка модели на адекватность

In [30]:
dummy = DummyClassifier(strategy='most_frequent').fit(features_train, target_train)
dummy_pred = dummy.predict(features_valid)
result = model_log.score(features_valid, target_valid)
print('Точность dummpy :', result)

model_random = RandomForestClassifier(random_state=12345, n_estimators=20, criterion='entropy', oob_score=True)
model_random.fit(features_train, target_train)
predictions = model_random.predict(features_valid)
result = model_random.score(features_valid, target_valid)
print('Точность случайного леса :', result)

Точность dummpy : 0.7325038880248833
Точность случайного леса : 0.8102643856920684


**Точность dummpy ниже чем точность случайного леса, и получается модель адекватна (хотя, я не знал вообще как проверять модели на адекватность =D**

## Вывод

### В данном исследовании была выявляна самая лучшая  модель, для определения рекомендаций по тарифу на основе данных о  количестве звонков, суммарной длительности звонков в минутах, количестве sms-сообщений и израсходованных Мб интернет-трафика. Самой точной показала себя модель случайного леса 0.80 на тестовой выборке (если округлить в большую сторону значение в 0.797) и 0.81 на валидационной выборке.